# 전처리

In [9]:
import os
import subprocess
from pathlib import Path

def preprocess_wav_directory(input_dir, output_dir, sample_rate=16000):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    for wav_file in input_dir.glob("*.wav"):
        output_file = output_dir / wav_file.name
        command = [
            "ffmpeg",
            "-y",  # overwrite without asking
            "-i", str(wav_file),
            "-ar", str(sample_rate),       # Resample to 16kHz
            "-ac", "1",                    # Convert to mono
            "-acodec", "pcm_s16le",        # 16-bit PCM
            str(output_file)
        ]
        subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    print(f"✅ 전처리 완료: {len(list(output_dir.glob('*.wav')))}개 파일이 {output_dir}에 저장되었습니다.")

# 예시 사용
input_path = 'input_wav'
output_path = "output_wav"
preprocess_wav_directory(input_path,output_path )


✅ 전처리 완료: 8개 파일이 output_wav에 저장되었습니다.


In [ ]:
!sudo reboot

import torch
import gc
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

gc.collect()


Mon May 26 00:48:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000001:00:00.0 Off |                  Off |
| N/A   39C    P8              9W /   70W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# STT

In [1]:
import azure.cognitiveservices.speech as speechsdk

def transcribe_speech(audio_path: str, speech_key: str, region: str) -> str:
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=region)
    speech_config.speech_recognition_language = "ko-KR"  # ✅ 한국어 설정

    audio_config = speechsdk.audio.AudioConfig(filename=audio_path)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("🎙️ STT 실행 중...")
    result = speech_recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("📝 인식된 텍스트:", result.text)
        return result.text
    else:
        print("❌ 음성 인식 실패:", result.reason)
        return ""

# 예시 실행
speech_key = "GG4wOUABvgwI2Im4ZFlBfX4H7N13tLCtJ8g6UHn2w5SV8n3A5HttJQQJ99BEACYeBjFXJ3w3AAAYACOGdyy5"
region = "eastus"
prompt_audio_path = "output_wav/노인남여_노인대화07_F_1526682663_63_수도권_실내_08352.wav"

prompt_text = transcribe_speech(prompt_audio_path, speech_key, region)


🎙️ STT 실행 중...
📝 인식된 텍스트: 나도 적지 않은 나이에 지금까지 성인병에 대해 공부하고 있어요.


# TTS

In [10]:
!python tools/llama/merge_lora.py \
    --lora-config r_8_alpha_16 \
    --base-weight checkpoints/fish-speech-1.5 \
    --lora-weight results/my_voice_project/checkpoints/step_000008000.ckpt \
    --output checkpoints/fish-speech-1.5-yth-lora-8000

2025-05-26 01:10:08.103 | INFO     | __main__:merge:23 - Merging checkpoints/fish-speech-1.5 and results/my_voice_project/checkpoints/step_000008000.ckpt into checkpoints/fish-speech-1.5-yth-lora-8000 with r_8_alpha_16
2025-05-26 01:10:08.180 | INFO     | __main__:merge:31 - Loaded lora model with config LoraConfig(r=8, lora_alpha=16, lora_dropout=0.01)
2025-05-26 01:10:21.410 | WARNING  | fish_speech.models.text2semantic.llama:from_pretrained:450 - No weight for embeddings.lora_A
2025-05-26 01:10:21.411 | WARNING  | fish_speech.models.text2semantic.llama:from_pretrained:450 - No weight for embeddings.lora_B
2025-05-26 01:10:21.411 | WARNING  | fish_speech.models.text2semantic.llama:from_pretrained:450 - No weight for codebook_embeddings.lora_A
2025-05-26 01:10:21.411 | WARNING  | fish_speech.models.text2semantic.llama:from_pretrained:450 - No weight for codebook_embeddings.lora_B
2025-05-26 01:10:21.411 | WARNING  | fish_speech.models.text2semantic.llama:from_pretrained:450 - No weigh

In [2]:
import torch

ckpt_path = "results/my_voice_project/checkpoints/step_000008000.ckpt"

ckpt = torch.load(ckpt_path, map_location="cpu", weights_only=False)

print("=== Checkpoint Keys ===")
for k in ckpt["state_dict"].keys():
    if "lora" in k:
        print(k)

print("\n총 LoRA 관련 key 개수:", len([k for k in ckpt["state_dict"].keys() if "lora" in k]))


=== Checkpoint Keys ===
model.embeddings.lora_A
model.embeddings.lora_B
model.codebook_embeddings.lora_A
model.codebook_embeddings.lora_B
model.layers.0.attention.wqkv.lora_A
model.layers.0.attention.wqkv.lora_B
model.layers.0.attention.wo.lora_A
model.layers.0.attention.wo.lora_B
model.layers.0.feed_forward.w1.lora_A
model.layers.0.feed_forward.w1.lora_B
model.layers.0.feed_forward.w3.lora_A
model.layers.0.feed_forward.w3.lora_B
model.layers.0.feed_forward.w2.lora_A
model.layers.0.feed_forward.w2.lora_B
model.layers.1.attention.wqkv.lora_A
model.layers.1.attention.wqkv.lora_B
model.layers.1.attention.wo.lora_A
model.layers.1.attention.wo.lora_B
model.layers.1.feed_forward.w1.lora_A
model.layers.1.feed_forward.w1.lora_B
model.layers.1.feed_forward.w3.lora_A
model.layers.1.feed_forward.w3.lora_B
model.layers.1.feed_forward.w2.lora_A
model.layers.1.feed_forward.w2.lora_B
model.layers.2.attention.wqkv.lora_A
model.layers.2.attention.wqkv.lora_B
model.layers.2.attention.wo.lora_A
model.lay

In [ ]:
from inference_module import init_engine, run_tts
import numpy as np
from scipy.io.wavfile import write as write_wav
engine = init_engine()

input_text = """
이 사진은 내가 스무 살 때, 처음 서울 구경 갔을 때 찍은 거야.
저기 옆에 있는 친구는 순이, 참 말도 많고 웃음도 많던 아이였지.
그날 남대문시장에서 산 하늘색 원피스를 아직도 기억해.
사진 속 나는 참 해맑은데, 그땐 세상이 다 설레고 신기했지.
요즘도 이 사진 보면 가끔 그 시절 냄새가 나는 것 같아.
"""
prompt_audio_path = "input_wav/test.wav"
prompt_text = '예시 프롬포트 입니다.'
# prompt_text = """책이 예술품처럼 예쁘게 만들어지는 시대인데, 내가 직접 타이핑하고, 교정 보고, 워드로 편집하고, 디자인까지 구상하며 만들었으니까 자서전의 겉모습은 평범하고 소박했어. 그래도 어머니께서는 일생이 고스란히 담긴 책을 받아들고는 무척 신기해하고 기뻐하셨어.
# 어머니는 배울 기회를 많이 갖지 못했지만 늘 책을 가까이하셨어. 삭막하고 딱딱한 시댁의 문화에, 공무원 남편의 박봉에 꿋꿋이 맞서 일생을 좀더 활기차게 사셨고 특히 강단 있고 요령 있게 세상과 맞짱 떠서 고단하던 가난을 끊어내셨어. 자식 삼 형제를 중산층으로 좀 만들어두시고, 책임 완수하신 아버님께서 홀가분한 마음으로 먼저 가시도록 잘 보내드린 후에 이제 신앙에 기대어 단정한 노년을 보내고 계셔
# 그러던 어머니께서 언젠가부터 내가 쓰지 않는다면서 가져다 드린 빈 공책에 연필로 꾹꾹 눌러가면서 지나온 이야기들을 기억나는 대로 조금씩 쓰기 시작하셨는데, 깔끔하게 정리해서 책으로 만들어 드리겠다며 원고를 가져다가 타이핑하기 시작했어"""

# 3. 추론 실행
sample_rate, waveform = run_tts(engine, input_text, prompt_audio_path, prompt_text)

# 4. float32 → int16로 변환하여 저장
waveform_int16 = (waveform * 32767).astype(np.int16)
write_wav("results/result.wav", sample_rate, waveform_int16)

print("✅ 생성 완료: result.wav")


/mnt/fish-work/fish-conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-29 02:33:32.655 | INFO     | fish_speech.models.text2semantic.inference:load_model:681 - Restored model from checkpoint
2025-05-29 02:33:32.656 | INFO     | fish_speech.models.text2semantic.inference:load_model:687 - Using DualARTransformer
/mnt/fish-work/fish-conda/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:445: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/mnt/fish-work/fish-conda/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:630: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` ins

KeyboardInterrupt: 

  4%|▍         | 310/7960 [00:22<09:19, 13.66it/s]
2025-05-29 02:33:57.109 | INFO     | fish_speech.models.text2semantic.inference:generate_long:860 - Generated 312 tokens in 23.18 seconds, 13.46 tokens/sec
2025-05-29 02:33:57.109 | INFO     | fish_speech.models.text2semantic.inference:generate_long:863 - Bandwidth achieved: 8.59 GB/s
2025-05-29 02:33:57.110 | INFO     | fish_speech.models.text2semantic.inference:generate_long:868 - GPU Memory used: 2.09 GB
2025-05-29 02:33:57.111 | INFO     | fish_speech.models.text2semantic.inference:generate_long:806 - Generating sentence 2/3 of sample 1/1
